In [6]:
from flask import Flask, url_for, render_template, request
import syspend
import numpy as np
import os
from utility import parse_config
from sentimental_analysis.bert.train import BertClassifier
from sentimental_analysis.bert.dataset import create_data_loader

In [7]:
app = Flask(__name__)

config_path = os.path.join(os.getcwd(), 'flask_app_config.yml')
config_file = parse_config(config_path)
model_name = config_file['flask']['model_name']
n_classes = int(config_file['flask']['num_classes'])
max_len = float(config_file['flask']['max_len'])
model_path = config_file['flask']['model_path']
THRESHOLD = int(config_file['flask']['THRESHOLD'])
MODEL =  BertClassifier(model_name, n_classes)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [8]:
#Routes - Creating a simple route
@app.route('/')
def index():
    return render_template("index.html")

In [9]:
@app.route('/predict', methods = ['GET', 'POST'])
def predict(): #Send data from front-end to backend then to front end
    if request.method == "POST":
        # Get the form data
        tweet = request.form["tweet"]
        test_dataloader = create_data_loader(model_name, 1,max_len, tweet,predict_only=True)
        pred_prob = predict(MODEL, test_dataloader)
        print(pred_prob)
        print(pred_prob[:, 1])
        preds = np.where(pred_prob[:, 1] > THRESHOLD, "Positive", "Negative")
        # Return the template with the form data
        return render_template("index.html", tweet = tweet,pred_prob = pred_prob, preds = preds)

In [10]:
if __name__ == '__main__':
    app.run(debug = True, use_reloader = True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/Users/gjj980/miniforge3/envs/voc_project/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/gjj980/miniforge3/envs/voc_project/lib/python3.8/site-packages/traitlets/config/application.py", line 1042, in launch_instance
    app.initialize(argv)
  File "/Users/gjj980/miniforge3/envs/voc_project/lib/python3.8/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
  File "/Users/gjj980/miniforge3/envs/voc_project/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 666, in initialize
    self.init_sockets()
  File "/Users/gjj980/miniforge3/envs/voc_project/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 307, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/Users/gjj980/miniforge3/envs

SystemExit: 1

/Users/gjj980/miniforge3/envs/voc_project/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:
!jupyter nbconvert --to script --no-prompt app.ipynb

[NbConvertApp] Converting notebook app.ipynb to script
[NbConvertApp] Writing 1631 bytes to app.py
